# Case study 1：添加 Prefetcher
# Case Study 1: Adding a Prefetcher

在这一部分，我们将通过一个在 XS-GEM5 中添加新预取器的示例，演示如何在 XS-GEM5 上实现并评估新的特性。

In this section, we present an example of adding a new prefetcher to XS-GEM5, demonstrating how to implement and evaluate new features on the XS-GEM5 platform.

我们先创建本节的工作目录。

First, we create the working directory for this section.

In [ ]:
%%bash
cd .. && source env.sh

mkdir -p ${WORK_DIR}/04-gem5/04-case-study-prefetcher

和 01-compile 以及 02-first-run 一样，我们先编译运行一份原始的 GEM5 作为基线。

Just like in 01-compile and 02-first-run, we first build and run a baseline version of GEM5.

In [ ]:
%%bash
cd .. && source env.sh
cd ${GEM5_HOME}

git reset --hard 30be231238bc0f9d3687efd59826748c02ad5f7b

# 编译
# compile
scons build/RISCV/gem5.opt --linker=mold -j `nproc`

GEM5=${WORK_DIR}/04-gem5/04-case-study-prefetcher/gem5-base.opt
cp build/RISCV/gem5.opt ${GEM5}

# 运行
# run
export LD_LIBRARY_PATH=${GEM5_HOME}/ext/dramsim3/DRAMsim3:${LD_LIBRARY_PATH}
export GCBV_REF_SO=$(get_asset gem5-precompile/riscv64-nemu-interpreter-c1469286ca32-so)

mkdir -p ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_base
cd ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_base

${GEM5} ${GEM5_HOME}/configs/example/xiangshan.py \
    --generic-rv-cpt=$(get_asset workload/mcf_12253_0.137576_.zstd) \
    --gcpt-restorer=$(get_asset workload/normal-gcb-restorer.bin) \
    -I 300000

我们准备好了一个实现 Despacito Stream Prefetcher 的 patch 文件 `04-DespacitoStream.patch`，现在可以将它应用到 GEM5 上并再次编译运行。


We have prepared a patch file '04-DespacitoStream.patch', which implements the Despacito Stream Prefetcher. Now we can apply it to GEM5 and rebuild the project.

In [ ]:
%%bash
cd .. && source env.sh
cd ${GEM5_HOME}

git reset --hard 30be231238bc0f9d3687efd59826748c02ad5f7b

# 应用 prefetcher patch
# use prefetcher patch
git config user.name "xiangshan-bootcamp"
git config user.email "xiangshan-bootcamp@example.com"
git am -3 ${XS_PROJECT_ROOT}/04-gem5/04-DespacitoStream.patch

# 编译
# compile
scons build/RISCV/gem5.opt --linker=mold -j `nproc`

GEM5=${WORK_DIR}/04-gem5/04-case-study-prefetcher/gem5-despacito.opt
cp build/RISCV/gem5.opt ${GEM5}

# 运行
# run
export LD_LIBRARY_PATH=${GEM5_HOME}/ext/dramsim3/DRAMsim3:${LD_LIBRARY_PATH}
export GCBV_REF_SO=$(get_asset gem5-precompile/riscv64-nemu-interpreter-c1469286ca32-so)

mkdir -p ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_despacito
cd ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_despacito

${GEM5} ${GEM5_HOME}/configs/example/xiangshan.py \
    --generic-rv-cpt=$(get_asset workload/mcf_12253_0.137576_.zstd) \
    --gcpt-restorer=$(get_asset workload/normal-gcb-restorer.bin) \
    -I 300000

最后，我们用一个简单的脚本对比一下添加预取器前后的差异。

Finally, we use a simple script to compare the differences before and after adding the prefetcher.

In [ ]:
%%bash
cd .. && source env.sh

echo -n "Baseline                IPC: "
cat ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_base/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2
echo -n "Despacito Stream        IPC: "
cat ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_despacito/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2
echo -n "Baseline         L1D Misses: "
cat ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_base/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2
echo -n "Despacito Stream L1D Misses: "
cat ${WORK_DIR}/04-gem5/04-case-study-prefetcher/result_despacito/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2